In [1]:
import os
import sys
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
#import seaborn as sn

In [ ]:
class NeuralNetwork:

    def __init__(self,input_layer,output_layer,loss,metrics):
        self.input_layer = input_layer
        self.output_layer = output_layer
        self.lossfunc = lossfunc[loss]




    

    def add_layer():

    def predict(self,input,):
        self.output.layer.forward()




    def train(self,input,target,epoche,eta,lam,n_batch,validation_split): #callbacks
        val_input = input[:,validation_split*input.shape[1]:]
        val_target = target[:,validation_split*input.shape[1]:]
        index = np.arange(len(input))
        np.shuffle(index)
        input_new = input[index]
        target_new = target[index]


        j = 0
        while j < epoche:
                
                np.shuffle(index)
                input_new = input[index]
                target_new = target[index]
                
                for k in range(input_new.shape[1]//n_batch):
                    self.input_layer.input = input_new[:,k*n_batch:(k+1)*n_batch]
                    self.target_layer.target = target_new[:,k*n_batch:(k+1)*n_batch]
                    self.output_layer.forward()
                    self.trainloss.append(self.output_layer.trainloss())
                    self.validloss.append(self.output_layer.validloss())
                    self.output_layer.backward()
                    self.output_layer.update_weights(eta, lam)
                j += 1

            


    

In [7]:
class Prova:

    def __init__(self):
        self.print_prova()
    
    def print_prova(self):
        print('ciao')

class Prova_sub(Prova):

    def __init__(self):
        Prova.__init__(self)
    
    def print_prova(self):
        print('ciao sub')

In [8]:
a = Prova()
b = Prova()
c = Prova_sub()

ciao
ciao
ciao sub


In [ ]:
input = Input()
hidden = Layer()
output = Layer()

model = NN(input,output,)
model.fit()